# Data Warehouse

The following script loads in data from two sources:
1. AirBnB data from the Washington, D.C. Metro Area [http://insideairbnb.com/get-the-data]
2. Crime data from the Washington, D.C. area [crimecards.dc.gov]

The airbnb data is for the last 4 quarters starting from December 15th 2021 to September 14th 2022. For each of the 4 quarters, we create 4 tables from the AirBnB data:
1. `listings`
2. `neighborhoods` (spelled `neighbourhoods` in the AirBnB website)
3. `reviews`
4. `calendar`

For the crime data, we create **TODO**

### Notes for running this notebook
* This notebook assumes that all source data .csv files are in the same directory as this script
* The September 14th, 2022 `listings.csv` data source included a `source` column that the other `listings.csv` files did not. We chose to drop this column for that quarter because `source` was simply where the data was scraped from and we don't plan on using that
* Order of import matters for `listings.csv` when using read_csv_auto. 11Jun, 14Sep, and 19Mar had a value, 558788119405795275, that couldn't be represented as an INTEGER (32-bit). If you import the December data first, then it defaults to storing this data as a BIGINT (64-bit), which seems to work. Alternatively, alter the column to BIGINT.

## Setup

We are using DuckDB version 0.5.1

In [4]:
import duckdb
import pandas as pd
import json
import os
import gzip
import shutil

In [77]:
con = duckdb.connect(database='ps6.duckdb', read_only=False)

## Add Data
Reference: https://duckdb.org/docs/api/python/overview

### 1. listings.csv

Team question: What was the reason for explicitly specifying the columns and types? Did the auto read csv not bring in the data correctly?

In [59]:

# con.execute('CREATE TABLE listings'
#             '(id BIGINT,'
#             'listing_url VARCHAR,'
#             'scrape_id BIGINT,'
#             'last_scraped DATE,'
#             'source VARCHAR,'
#             'name VARCHAR,'
#             'description VARCHAR,'
#             'neighborhood_overview VARCHAR,'
#             'host_id INTEGER,'
#             'host_url VARCHAR,'
#             'host_name VARCHAR,'
#             'host_since DATE,'
#             'host_location VARCHAR,'
#             'host_about VARCHAR,'
#             'host_response_time VARCHAR,'
#             'host_response_rate VARCHAR,'
#             'host_acceptance_rate VARCHAR,'
#             'host_is_superhost BOOLEAN,'
#             'host_neighbourhood VARCHAR,'
#             'host_listings_count INTEGER,'
#             'host_total_listings_count INTEGER,'
#             'host_verifications VARCHAR,'
#             'host_has_profile_pic BOOLEAN,'
#             'host_identity_verified BOOLEAN,'
#             'neighbourhood_group VARCHAR,'
#             'neighbourhood VARCHAR,'
#             'neighbourhood_cleansed VARCHAR,'
#             'neighbourhood_group_cleansed VARCHAR,'
#             'latitude DOUBLE,'
#             'longitude DOUBLE,'
#             'property_type VARCHAR,'
#             'room_type VARCHAR,'
#             'accommodates INTEGER,'
#             'bathrooms VARCHAR,'
#             'bathrooms_text VARCHAR,'
#             'bedrooms INTEGER,'
#             'beds INTEGER,'
#             'amenities VARCHAR,'
#             'price INTEGER,'
#             'minimum_nights INTEGER,'
#             'maximum_nights INTEGER,'
#             'minimum_minimum_nights INTEGER,'
#             'maximum_minimum_nights INTEGER,'
#             'minimum_maximum_nights INTEGER,'
#             'maximum_maximum_nights INTEGER,'
#             'minimum_nights_avg_ntm DOUBLE,'
#             'maximum_nights_avg_ntm DOUBLE,'
#             'calendar_updated VARCHAR,'
#             'has_availability BOOLEAN,'
#             'availability_30 INTEGER,'
#             'availability_60 INTEGER,'
#             'availability_90 INTEGER,'
#             'availability_365 INTEGER,'
#             'calendar_last_scraped DATE,'
#             'number_of_reviews INTEGER,'
#             'number_of_reviews_ltm INTEGER,'
#             'number_of_reviews_l30d INTEGER,'
#             'first_review DATE,'
#             'last_review DATE,'
#             'review_scores_rating DOUBLE,'
#             'review_scores_accuracy DOUBLE,'
#             'review_scores_cleanliness DOUBLE,'
#             'review_scores_checkin DOUBLE,'
#             'review_scores_communication DOUBLE,'
#             'review_scores_location DOUBLE,'
#             'review_scores_value DOUBLE,'
#             'license VARCHAR,'
#             'instant_bookable BOOLEAN,'            
#             'calculated_host_listings_count INTEGER,'
#             'calculated_host_listings_count_entire_homes INTEGER,'
#             'calculated_host_listings_count_private_rooms INTEGER,'
#             'calculated_host_listings_count_shared_rooms INTEGER,'
#             'reviews_per_month DOUBLE,'
#             'PRIMARY KEY(calendar_last_scraped, id))')

In [60]:
#Original

# con.execute("DROP TABLE IF EXISTS listings")
# con.execute("CREATE TABLE listings AS SELECT * FROM read_csv_auto('14Sep22 listings.csv');")
# con.execute("INSERT INTO listings SELECT * FROM read_csv_auto('11Jun22 listings.csv');")
# con.execute("INSERT INTO listings SELECT * FROM read_csv_auto('15Dec21 listings.csv');")
# con.execute("INSERT INTO listings SELECT * FROM read_csv_auto('19Mar22 listings.csv');")



In [78]:
con.execute('DROP TABLE IF EXISTS all_listings;')
con.execute("CREATE TABLE all_listings AS SELECT * FROM read_csv_auto('14Sep22 listings 2.csv');")
con.execute("ALTER TABLE all_listings DROP source;")
con.execute("INSERT INTO all_listings SELECT * FROM read_csv_auto('11Jun22 listings 2.csv');")
con.execute("INSERT INTO all_listings SELECT * FROM read_csv_auto('15Dec21 listings 2.csv');")
con.execute("INSERT INTO all_listings SELECT * FROM read_csv_auto('19Mar22 listings 2.csv');")


#### Data Cleanup
Change the price column to a numerical type.


In [79]:
# Pandas version
all_listings_df = con.execute("SELECT * from all_listings").df()
all_listings_df.price = all_listings_df.price.replace('\$|,', '', regex=True).astype(float)
con.execute('DROP TABLE IF EXISTS all_listings;')
con.execute("CREATE TABLE all_listings AS SELECT * FROM all_listings_df")

In [ ]:
# SQL command version
# con.execute('alter table all_listings rename price to price_text;')

# con.execute('alter table all_listings add column price double;')
# con.execute('''update all_listings set price = regexp_replace(price_text, '[$,]', '', 'g');''')

#### Create a view for just the latest listings

The all_listings table includes multiple entries for the same listing. We may want to just deal with one listing for the whole year, taking the latest data. This would weight each listing the same.  

First, we need to determine the latest listing for each listing (e.g. when was the latest data scrape for listing 3686?)

In [80]:
con.execute('drop view if exists last_scraped')
con.execute('create view last_scraped as select id, max(calendar_last_scraped) as last_scraped from all_listings group by id;')

Now, we'll create a view called latest_listings that only takes the latest listing data for each listing.

In [81]:
con.execute('drop view if exists latest_listings')
con.execute('create view latest_listings as select all_listings.* from all_listings inner join last_scraped on all_listings.id = last_scraped.id and all_listings.calendar_last_scraped = last_scraped.last_scraped; ')

How many listings are there compared to the entire data set?

In [82]:
con.execute("select count(id) from latest_listings;")
print(con.fetchall())

[(10560,)]


There were 28076 total rows in the all_listings data.

con.execute("select count(id) from latest_listings;")
print(con.fetchall())

The latest_listings view, which only has one row per listing, has just 10560 rows, less than half of the original data set.

### 2. neighborhood.csv

In [13]:
con.execute('DROP TABLE IF EXISTS neighborhoods;')
con.execute('CREATE TABLE neighborhoods'
            '(neighbourhood_group VARCHAR,'
            'neighbourhood VARCHAR PRIMARY KEY,'
            'type_outer VARCHAR,'
            'feature_type VARCHAR,'
            'geometry_type VARCHAR,'
            'coordinates DOUBLE[][])')

In [14]:
df = pd.read_csv("11Jun22 neighbourhoods.csv")

In [15]:
names = ["11Jun22 neighbourhoods.geojson", "14Sep22 neighbourhoods.geojson", "15Dec21 neighbourhoods.geojson", "19Mar22 neighbourhoods.geojson"]
total = pd.DataFrame()
last = pd.DataFrame(columns=["type_outer", "feature_type", "geometry_type", "coordinates", "neighborhood", "neighborhood group"])
diff = False
for n in names:
    with open(n, 'r') as f:
        data = json.load(f)
    
        row = []
        type_outer = ""
        type_outer = data["type"]
        features = []
        feature_type = ""
        geometry, properties = [], []
        for y in range(0, len(data["features"])):
            features = data["features"][y]
            feature_type = features["type"]
            geometry = features["geometry"]
            properties = features["properties"]
            geometry_type, neigh, neigh_group = "", "", ""
            coords = []
            geometry_type = geometry["type"]
            coords = geometry["coordinates"][0][0]
            neigh = properties["neighbourhood"]
            neigh_group = properties["neighbourhood_group"]
            
            if len(last) == 0:
                row.append([type_outer, feature_type, geometry_type, coords, neigh, neigh_group])
                diff = True
                
            elif list(last[last["neighborhood"] == neigh]["coordinates"])[0] != list(coords):
                lastcoords = list(last[last["neighborhood"] == neigh]["coordinates"])[0]
                diff = True
                row.append([type_outer, feature_type, geometry_type, coords, neigh, neigh_group])
        
        if diff == True:
            last = pd.DataFrame(row, columns=["type_outer", "feature_type", "geometry_type", "coordinates", "neighborhood", "neighborhood group"])
            json_df = pd.DataFrame(row, columns=["type_outer", "feature_type", "geometry_type", "coordinates", "neighborhood", "neighborhood group"])
            diff = False
            
            pd.concat([df, pd.read_csv(n.replace("geojson", "csv"))])
            df.drop_duplicates()
            
            new_df = df.join(json_df.set_index("neighborhood"), on="neighbourhood")
            
            total = pd.concat([total, new_df])

In [16]:
total = total.drop(['neighborhood group'], axis=1)

In [17]:
con.execute("INSERT INTO neighborhoods SELECT * FROM total;")

### 3. reviews.csv

In [18]:
con.execute('DROP TABLE IF EXISTS reviews;')
con.execute('CREATE TABLE reviews'
            '(listing_id BIGINT,'
            'id BIGINT PRIMARY KEY,'
            'date DATE,'
            'reviewer_id INTEGER,'
            'reviewer_name VARCHAR,'
            'comments VARCHAR)')

In [19]:
reviews_df = pd.read_csv('14Sep22 reviews 2.csv')
pd.concat([reviews_df, pd.read_csv('11Jun22 reviews 2.csv')])
pd.concat([reviews_df, pd.read_csv('15Dec21 reviews 2.csv')])
pd.concat([reviews_df, pd.read_csv('19Mar22 reviews 2.csv')])
reviews_df.drop_duplicates(subset=['id'])

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3571,36234,2010-04-18,81912,Jon,"Don's apartment is comfortable, clean and well..."
1,3571,51518,2010-06-10,112316,David,Don was a great host. He went out of his way t...
2,3571,132238,2010-11-02,255952,Diana,This place was great! Don was extremely helpfu...
3,3571,326887,2011-06-21,637695,Monica,This condo was in a great location for touring...
4,3571,368063,2011-07-12,448224,Florence,"The apartment was very clean, comfortable and ..."
...,...,...,...,...,...,...
321204,705262593266969561,709423398399592317,2022-09-05,464219597,Charlene,"Beautiful apartment, very safe"
321205,705307910480790712,709393953785783433,2022-09-05,43384449,Rainer,Great quiet getaway in DC in residential neigh...
321206,705307910480790712,713759649218625398,2022-09-11,261620474,Ricardo,"Nice and cozy stay, easy to find with self che..."
321207,708911697866238625,713003106344715260,2022-09-10,225974179,Johnny,Very accommodating. Great space with lots of a...


In [20]:
con.execute("INSERT INTO reviews SELECT * FROM reviews_df;")

### 4. calendar.csv

In [21]:
# function to unzip the file for cleaning and rename it with a unique id
def unzipcalendar(file,uniqueId):
    new_file = 'calendar'+uniqueId+'.csv'
    with gzip.open(file, 'r') as f_in, open(new_file, 'wb') as f_out:
      shutil.copyfileobj(f_in, f_out)
    print(new_file)

In [22]:
# cleans the commas out of prices and adjusted_prices column and saves the data as a float back into the csv
def cleanPrices(filename):
        df = pd.read_csv(filename)
        df['price'] = df['price'].str.replace(',', '').astype(float)
        df['adjusted_price'] = df['adjusted_price'].str.replace(',', '').astype(float)
        df.to_csv(filename, encoding='utf-8', index=False)

In [23]:
# function to clean the csv and remove '$' from the price data so it can be imported as int.
# @arg filename (file.csv)
# @arg match (string, e.g. '$', 'text')
def cleancsv(filename, match):
    original_file = filename
    temp_file = "temp.csv"
    
    string_to_delete = [match]
    with open(original_file, "r") as input:
        with open(temp_file, "w") as output:
            for line in input:
                for word in string_to_delete:
                    line = line.replace(word, "")
                output.write(line)
    os.replace('temp.csv', filename)
    cleanPrices(filename)
    print(filename, ' cleaned')

Unzipping and cleaning the files before we import into duckdb

In [99]:
unzipcalendar('15Dec21 calendar.csv.gz',"12_2021")
unzipcalendar('19Mar22 calendar.csv.gz',"03_2022")
unzipcalendar('11Jun22 calendar.csv.gz',"06_2022")
unzipcalendar('14Sep22 calendar.csv.gz',"09_2022")

calendar12_2021.csv
calendar03_2022.csv
calendar06_2022.csv
calendar09_2022.csv


In [100]:
cleancsv('calendar12_2021.csv','$')
cleancsv('calendar03_2022.csv','$')
cleancsv('calendar06_2022.csv','$')
cleancsv('calendar09_2022.csv','$')

calendar12_2021.csv  cleaned
calendar03_2022.csv  cleaned
calendar06_2022.csv  cleaned
calendar09_2022.csv  cleaned


### Import csv into duckdb

In [24]:
con.execute('CREATE OR REPLACE TABLE calendar'
            '(listing_id BIGINT,'
            'date DATE,'
            'available VARCHAR,'
            'price INTEGER,'
            'adjusted_price INTEGER,'
            'minimum_nights INTEGER,'
            'maximum_nights INTEGER);')
con.execute("COPY calendar FROM 'calendar12_2021.csv' (AUTO_DETECT TRUE);")

In [25]:
con.execute("INSERT INTO calendar SELECT * FROM read_csv('calendar09_2022.csv', delim=',', header=True, columns={'listing_id': 'BIGINT', 'date': 'DATE', 'available': 'VARCHAR', 'price': 'INTEGER', 'adjusted_price': 'INTEGER', 'minimum_nights': 'INTEGER', 'maximum_nights': 'INTEGER'});") 

In [26]:
con.execute("INSERT INTO calendar SELECT * FROM read_csv_auto('calendar03_2022.csv');")
con.execute("INSERT INTO calendar SELECT * FROM read_csv_auto('calendar06_2022.csv');")

### 5. crime.csv

In [27]:
con.execute('DROP TABLE IF EXISTS crimes;')


In [28]:

# con.execute('CREATE TABLE crimes'
#             '(neighborhood_cluster VARCHAR,'
#             'census_tract INTEGER,'
#             'offensegroup VARCHAR,'
#             'longitude DOUBLE,'
#             'end_date VARCHAR,' # TODO: should definitely be DATE
#             'offense_text VARCHAR,'
#             'shift VARCHAR,'
#             'yblock DOUBLE,'
#             'district INTEGER,'
#             'year INTEGER,'
#             'ward INTEGER,'
#             'offensekey VARCHAR,'
#             'bid VARCHAR,'
#             'sector VARCHAR,'
#             'ucr_rank INTEGER,'
#             'psa INTEGER,'
#             'block_group VARCHAR,'
#             'voting_precint VARCHAR,'
#             'xblock DOUBLE,'
#             'block VARCHAR,'
#             'start_date VARCHAR,' # TODO: should definitely be DATE
#             'ccn INTEGER,'
#             'offense VARCHAR,'
#             'octo_record_id VARCHAR,'
#             'anc VARCHAR,'
#             'report_dat VARCHAR,' # TODO: should definitely be DATE
#             'method VARCHAR,'
#             'location VARCHAR,' # TODO: should be DOUBLE[][] but also this column is redundant
#             'latitude DOUBLE)')

In [29]:
#con.execute("INSERT INTO crimes SELECT * FROM read_csv('dc-crimes-search-results-V2.csv', dateformat='%m/%d/%Y, %H:%M:%S %p', AUTO_DETECT=TRUE);")

In [32]:
con.execute("CREATE TABLE crimes as SELECT * FROM read_csv('dc-crimes-search-results-V2.csv', dateformat='%m/%d/%Y, %H:%M:%S %p', AUTO_DETECT=TRUE);")

In [33]:
con.execute("SELECT COUNT(*) FROM crimes")
print(con.fetchall())

[(27611,)]


In [34]:
con.execute("DESCRIBE")
print(con.fetchall())

[('all_listings', ['accommodates', 'amenities', 'availability_30', 'availability_365', 'availability_60', 'availability_90', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'calendar_last_scraped', 'calendar_updated', 'description', 'first_review', 'has_availability', 'host_about', 'host_acceptance_rate', 'host_has_profile_pic', 'host_id', 'host_identity_verified', 'host_is_superhost', 'host_listings_count', 'host_location', 'host_name', 'host_neighbourhood', 'host_picture_url', 'host_response_rate', 'host_response_time', 'host_since', 'host_thumbnail_url', 'host_total_listings_count', 'host_url', 'host_verifications', 'id', 'instant_bookable', 'last_review', 'last_scraped', 'latitude', 'license', 'listing_url', 'longitude', 'maximum_maximum_nights', 'maximum_minimum_nights', 'maximum_nights', 'maximum_nights_

#### Aggregate Crime Table
For each neighborhood, this counts the number of offenses per type.  Of particular importance is the number of violent crimes in a neighborhood.

Feel free to add to this.

In [83]:
con.execute('drop view if exists neighborhood_crimes')
con.execute('create view neighborhood_crimes as Select neighborhood_name, '
        'count(neighborhood_name) FILTER (WHERE offense = \'homicide\') as homicides, '
        'count(neighborhood_name) FILTER (WHERE offense = \'robbery\') as robberies, '
        'count(neighborhood_name) FILTER (WHERE offense = \'assault w/dangerous weapon\') as assaults, '
        'count(neighborhood_name) FILTER (WHERE offense = \'theft f/auto\') as theft_from_auto, '
        'count(neighborhood_name) FILTER (WHERE offense = \'theft/other\') as other_thefts, '
        'count(neighborhood_name) FILTER (WHERE offense = \'motor vehicle theft\') as vehicle_theft, '
        'count(neighborhood_name) FILTER (WHERE offense = \'burglary\') as burglaries, '
        'count(neighborhood_name) FILTER (WHERE offense = \'sex abuse\') as sex_abuses, '
        'count(neighborhood_name) FILTER (WHERE offense = \'arson\') as arsons,'
        'count(neighborhood_name) FILTER (WHERE offensegroup = \'violent\') as violent_crimes,'
        'count(neighborhood_name) FILTER (WHERE offensegroup = \'property\') as property_crimes,'
        'count(neighborhood_name) as total_crimes '
        'from crimes group by neighborhood_name;')



### Additional Views


#### Full Latest Listings

This joins the latest listings to the neighborhood crime statistics.

In [84]:
# needs to be joined to crime aggregate table
con.execute('drop view if exists full_latest_listings')
con.execute('create view full_latest_listings as select * from latest_listings left join neighborhood_crimes on latest_listings.neighbourhood_cleansed = neighborhood_crimes.neighborhood_name;')

## Close the connection
The duckdb file is locked until now... You'll have to close it to run CLI commands.

In [85]:
con.close()